In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] ='0'


import spacy
nlp = spacy.load('fr_dep_news_trf')

In [ ]:
import sys
sys.path.append('..')
from extract_corpus import get_coref_corpus

corpus = get_coref_corpus()

In [ ]:
doc = nlp("Mme JOUDKIA est jolie. Elle mange des fleurs. Monsieur LEBORGNE est malade. Il est à l'hopital.")
for tok in doc:
  print(f"{tok.text:<10} {tok.dep_:<10} {tok.pos_}")

In [ ]:

# for year in corpus:
#   print(year)

for year in ["1814"]:
# for year in corpus:
  year_text = "\n".join([chunk.text for chunk in corpus[year]]) #le but c'est de concatener le texte de chaque chunk de 1917
  doc = nlp(year_text)
  adjectifs = [token for token in doc if token.pos_ == "ADJ"]
  for adj in adjectifs:
    for child in adj.children:
        if child.dep_ == "nsubj":
          subj = None
          for chunk in doc.noun_chunks:
            if (chunk[0].i <= child.i) and (child.i <= chunk[-1].i):
              subj = chunk
          if not subj:
            subj = child
          print(f"{adj.text:<20} || {subj.text:<20} || {subj.}")
          print("=================")


In [ ]:
nlp = spacy.load("fr_core_news_lg")

def compute_adjectives_subjects(doc):
    local_adj_subj = {}
    for ent in doc.ents:
        if ent.label_ == "PER":
            local_adj_subj[ent] = []

    for ent, adjs in local_adj_subj.items():
        for token in ent:
            if token.head.pos_ == "ADJ":
                adjs.append(token.head)
    
    for subject, adjs in local_adj_subj.items():
        if subject.text not in adjectives_subjects:
            adjectives_subjects[subject.text] = []
        adjectives_subjects[subject.text].extend(adjs)


adjectives_subjects = {}
for year in corpus:
    print("YEAR = ", year)
    year_text = "\n".join([chunk.text for chunk in corpus[year]]) #le but c'est de concatener le texte de chaque chunk de 1917
    doc = nlp(year_text)
    compute_adjectives_subjects(doc)



In [102]:
index=0
for subject, adjs in adjectives_subjects.items():
    if (len(adjs)>0):
        index+=1
        print(f"{subject} -> {adjs}")


Raphaël Duflos -> [Grand, Grand, Grand, Grand, Grand, Grand, Grand, Grand, Grand]
Silvain -> [excessif, agréées.[10, 1er]
M. -> [supérieur, prêts, impossible, libre, autre, confirmant, agréable, formel, première, autre, remercie, remercie, vive, remercie, Récapitulatif, désire, droit, remercie, désire, droit, prudent, niable, niable, première, précises, tenante, royal, urgent, formel, 1er, surintendant, agréable, précises, recueuillies, attendu, Française, utile, positives, ignorées, impossible, aimable, embarrassé]
M. Raphaël Duflos -> [douzième]
M. Silvain -> [celui]
Mme -> [première, dues, relatives, relatives, simple, nécessaires, nécessaires, satisfait, douzième, semblable, relative]
M. Delaunay -> [autres, retraité, indispensable, heureux, difficile, heureux]
Auguste Lançon -> [temporaire]
M. Le Bargy -> [relatives, relatives, relatives]
M. Corron -> [autre]
Talma -> [relative, ILLisible, postérieurs, élevé, élevée, élevée, relatif]
M. de Max -> [relative]
Monsieur -> [heureux, s